In [1]:
# from google.colab import drive
# drive.mount('/content/drive')


# Using Weights&Bias

In [1]:
# !pip install wandb

# !pip install pytorch-lightning

In [7]:
import wandb
wandb.login(key='ebdb46b0fb5d52411052955d3879ce00006895f1')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kaifzheng. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kaife/.netrc


True

# Neural network for distortion project

## import packages

In [8]:
import torch
from functools import partial
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader, TensorDataset,IterableDataset,Dataset
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.ndimage import gaussian_filter1d
from scipy.signal.windows import triang
from scipy.ndimage import convolve1d
from collections import Counter
import torch.nn.functional as F
from pytorch_lightning.loggers import WandbLogger
# demonstrate data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
import joblib
from util.data_preparation.data_loader import *
from util.data_preparation.data_helper import *
from util.data_preparation.data_mod import *
from util.NN_architecture import *
from util.NN_model import *



In [9]:
import math
import wandb
# sweep
sweep_config = {
    "method":'bayes',

}

metric = {
    'name':'val_loss_epoch',
    'goal':'minimize'
}

sweep_config['metric']=metric

parameters = {
     'batch_size':{
         'value': 128
         #'value':64
     },
    #  'delta':{
    #      'distribution':'uniform',
    #      'min':0.01,
    #      'max':1
    #  },
    # 'delta':{
    #     'value':0.2
    # },
    'output1':{
        'values':[16,32,64,128]
    },
    'kernel1':{
            'values':[3,5,7,9,11]
    },
    'dropout1':{
        'distribution':'uniform',
        'min':0,
        'max':0.3
    },
    'output2':{
        'values':[32,64,128,256]
    },
    'kernel2':{
            'values':[3,5,7,9,11]
    },
    'dropout2':{
        'distribution':'uniform',
        'min':0,
        'max':0.3
    },
    'output3':{
        'values':[16,32,64,128]
    },
    'kernel3':{
            'values':[3,5,7,9,11]
    },
    'dropout3':{
        'distribution':'uniform',
        'min':0,
        'max':0.3
    },
    'output4':{
        'values':[16,32,64,128]
    },
    'kernel4':{
            'values':[3,5,7,9,11]
    },
    'dropout4':{
        'distribution':'uniform',
        'min':0,
        'max':0.3
    },
     'l_nn':{
        'values':[50,100,200,300,150,250]
     },
     'kmesh_points':{
       'values':[150,200,250,300,400]  
     },
     'training_noise_level':{
        #  'values':[0.01,0.02,0.05,0.07,0.1]
        'value': 0.1
     }
    #  'slope':{
    #      'distribution':'uniform',
    #      'min':0.01,
    #      'max':1
}




# add some other values
parameters.update(
    {
        'epochs':{
            'value':50
        },
        'learning_rate':{
         'value':3
     },
    }
)

sweep_config['parameters']=parameters

In [10]:
#step2 sweep
sweep_id = wandb.sweep(sweep_config,project="CNN_gr_10182023")


Create sweep with ID: 3y25meuj
Sweep URL: https://wandb.ai/kaifzheng/CNN_gr_10182023/sweeps/3y25meuj


In [11]:

def train(config=None):
  with wandb.init(config=config):
      config = wandb.config
      wandb_logger = WandbLogger(name='CNN_XANES',project="CNN_gr_12252023",log_model=True)
      parameters={'output1':config.output1,
            'kernel1':config.kernel1,
            'dropout1':config.dropout1,
            'output2':config.output2,
            'kernel2':config.kernel2,
            'dropout2':config.dropout2,
            'output3':config.output3,
            'kernel3':config.kernel3,
            'dropout3':config.dropout3,
            'output4':config.output4,
            'kernel4':config.kernel4,
            'dropout4':config.dropout4,
            'linear1':config.l_nn
            }
      data_mod=NNGrDataModule(data_dir='dataset_partial/',
                spectra_data_path='output_ave_test_kx.csv',
                gr_data_path=['gr.csv'],
                kmesh_path='kmesh.txt',
                rmesh_path='rmesh.txt',
                kmesh_bound=[1.5,8.5],
                rmesh_bound=[1.5,5.5],
                k_win_range=[2,8],
                kmesh_points=config.kmesh_points,
                rmesh_points=400,
                dx=2,
                kweight=2,
                window_func='hanning',
                training_noise_level=config.training_noise_level,
                batch=config.batch_size,
                concate=False,
                partial_gr=False
                )
      data_mod.prepare_data_train()
      data_mod.setup(stage='fit')
      train_loader=data_mod.train_dataloader()
      val_loader=data_mod.val_dataloader()
      rmesh_new=data_mod.rmesh_new
      kmesh_new=data_mod.kmesh_new

      checkpoint_callback = ModelCheckpoint(monitor="val_loss_epoch", mode="min")
      #afunc:tanh,ReLU
      net=GEXNN_AT_para(feature_len=len(kmesh_new),gr_len=data_mod.rmesh_points,config_para=parameters)
      model=NN_model(net,nn.L1Loss(),lr=config.learning_rate)
      steps_per_epoch = int(10000 / config.batch_size)
      trainer = pl.Trainer(max_epochs=config.epochs,limit_train_batches=steps_per_epoch,limit_val_batches=steps_per_epoch,logger=wandb_logger,callbacks=[checkpoint_callback,pl.callbacks.EarlyStopping(monitor='val_loss', patience=50, min_delta=0.0001, mode='min',verbose=False)])#,callbacks=[pl.callbacks.EarlyStopping(monitor='val_loss', patience=100, min_delta=0.0001, mode='min',verbose=False)]
      trainer.fit(model, train_loader, val_loader)

## Build Neural Network

In [12]:

# from torchsummary import summary
# model=MyModel(0.01,12,16,14,0.1,0.1,0.1,3,3,2,3,2,3,100)
# summary(model,input_size=(1,300),batch_size=32,device='cpu')

In [13]:
#from torch.optim.lr_scheduler import ReduceLROnPlateau
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1,momentum=0.6)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 100,min_lr=1e-5,threshold=1e-8,factor=0.8)
#predict = model(x)
#loss_func = torch.nn.MSELoss()


In [14]:
torch.cuda.memory_summary(device=None, abbreviated=False)
torch.cuda.empty_cache()
# torch.cuda.max_memory_allocated(device=None)
torch.cuda.memory_summary(device=0, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [15]:
wandb.agent(sweep_id,train,count=500)

wandb: Agent Starting Run: ixcaxtto with config:
wandb: 	batch_size: 128
wandb: 	dropout1: 0.2848260249975084
wandb: 	dropout2: 0.26843037079491283
wandb: 	dropout3: 0.18897781302235145
wandb: 	dropout4: 0.14980350077527635
wandb: 	epochs: 50
wandb: 	kernel1: 5
wandb: 	kernel2: 7
wandb: 	kernel3: 11
wandb: 	kernel4: 11
wandb: 	kmesh_points: 150
wandb: 	l_nn: 150
wandb: 	learning_rate: 3
wandb: 	output1: 128
wandb: 	output2: 32
wandb: 	output3: 64
wandb: 	output4: 32
wandb: 	training_noise_level: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


c:\Users\kaife\anaconda3\envs\NN_work\lib\site-packages\pytorch_lightning\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


bad data:360


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A3000 12GB Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\kaife\anaconda3\envs\NN_work\lib\site-packages\lightning\pytorch\core\optimizer.py:289: RuntimeWarning: A `OneCycleLR` scheduler is using 'interval': 'epoch'. Are you sure you didn't mean 'interval': 'step'?
  rank_zero_warn(

  | Name          | Type          | Params
------------------------------------------------
0 | net           | GEXNN_AT_para | 858 K 
1 | loss          | L1Loss        | 0     
2 | train_loss    |

Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
np.ceil(12.5)

In [ ]:
#def test(model, test_loader):
#  model.eval()

#  with torch.no_grad():
#    correct,total=0,0
#    for data,labels in test_loader:
##      data = data.to(device)
 #     label = label.to(device)
 #     output = model(data)


 #   wandb.log("")